<a href="https://colab.research.google.com/github/mallick20/LearnX-Sales-Forecasting/blob/main/CourseSales_Forecasting_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Required Packages

In [2]:
from time import time
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#print(os.listdir("../input"))
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [3]:
train = pd.read_csv("/content/drive/MyDrive/CourseSales_Forecasting/train.csv")
test = pd.read_csv("/content/drive/MyDrive/CourseSales_Forecasting/test_6hcImfB.csv")


In [4]:
print(train.shape)
train.head()

(512087, 11)


,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
0,1,1,1,Development,Course,0,1,1,11004,0.007,81
1,2,2,1,Development,Course,0,0,1,13650,0.007,79
2,3,3,1,Development,Course,0,0,1,11655,0.007,75
3,4,4,1,Development,Course,0,0,1,12054,0.007,80
4,5,5,1,Development,Course,0,0,1,6804,0.007,41


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512087 entries, 0 to 512086
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID                  512087 non-null  int64  
 1   Day_No              512087 non-null  int64  
 2   Course_ID           512087 non-null  int64  
 3   Course_Domain       512087 non-null  object 
 4   Course_Type         512087 non-null  object 
 5   Short_Promotion     512087 non-null  int64  
 6   Public_Holiday      512087 non-null  int64  
 7   Long_Promotion      512087 non-null  int64  
 8   User_Traffic        512087 non-null  int64  
 9   Competition_Metric  510323 non-null  float64
 10  Sales               512087 non-null  int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 43.0+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  36000 non-null  int64  
 1   Day_No              36000 non-null  int64  
 2   Course_ID           36000 non-null  int64  
 3   Course_Domain       36000 non-null  object 
 4   Course_Type         36000 non-null  object 
 5   Short_Promotion     36000 non-null  int64  
 6   Public_Holiday      36000 non-null  int64  
 7   Long_Promotion      36000 non-null  int64  
 8   Competition_Metric  35880 non-null  float64
dtypes: float64(1), int64(6), object(2)
memory usage: 2.5+ MB


Exploratory Data Analysis

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
# Null Values in each column
train.isna().sum()

In [ ]:
for col in train.columns:
  print(col,train[col].nunique())

In [ ]:
# Course Domain
print(train["Course_Domain"].value_counts())
sns.countplot(x="Course_Domain",data=train)

In [ ]:
#Course Type
plt.figure(1)
sns.countplot(x="Course_Type",data=train)

plt.figure(2)
sns.countplot(x="Course_Type", data=test)

In [ ]:
#Public Holiday
print(train["Public_Holiday"].value_counts())
plt.figure(1)
sns.countplot(x="Public_Holiday",data=train)
plt.figure(2)
sns.countplot(x="Public_Holiday", data=test)

In [ ]:
# Short and Long Promotion
df1 = train[(train.Short_Promotion==1) & (train.Long_Promotion==0)]
df2 = train[(train.Short_Promotion==0) & (train.Long_Promotion==1)]
df3 = train[(train.Short_Promotion==1) & (train.Long_Promotion==1)]
df4 = train[(train.Short_Promotion==0) & (train.Long_Promotion==0)]


for df in (df1,df2,df3,df4):
  print(df.shape[0],df["Sales"].mean())

plt.figure(figsize=(6,6))
plt.subplot(2,1,1)
sns.countplot(x="Short_Promotion",data=train)
plt.subplot(2,1,2)
sns.countplot(x="Long_Promotion",data=train)

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
sns.displot(data = train, x = "Competition_Metric", kind = "kde")
plt.subplot(2,1,2)
sns.displot(data = test, x = "Competition_Metric", kind = "kde")


In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
sns.boxplot(x="Competition_Metric", data=train)
plt.subplot(2,1,2)
sns.boxplot(x="Competition_Metric", data=test)


In [ ]:
print(train["Day_No"].values)
print(test["Day_No"].values)

In [ ]:
print(train['Course_ID'].unique())
print(test['Course_ID'].unique())

In [ ]:
test.isna().sum()

In [ ]:
#target variable
sns.distplot(train['Sales'])

In [ ]:
# it is skewed hence while performing model,log1p transformation is applied

In [ ]:
# sales of course_ID 1 full timne
strain = train[train.Sales>0]
strain.loc[strain['Course_ID']==1 ,['Day_No','Sales']].plot(x='Day_No',y='Sales',title='Course_ID 1',figsize=(16,4))


In [ ]:
# sales of course_ID 2  full timne
strain = train[train.Sales>0]
strain.loc[strain['Course_ID']==2 ,['Day_No','Sales']].plot(x='Day_No',y='Sales',title='Course_ID 2',figsize=(16,4))


In [ ]:
#sales of course_id 1 ,day no 240 vs 605
strain = train[train.Sales>0]
strain.loc[strain['Course_ID']==1 ,['Day_No','Sales']]     .plot(x='Day_No',y='Sales',title='Course_ID 1',figsize=(8,2),xlim=[240,605])
strain.loc[strain['Course_ID']==1 ,['Day_No','Sales']]     .plot(x='Day_No',y='Sales',title='Course_ID 1',figsize=(8,2),xlim=[240,605])

Feature Engineering

In [7]:
# Creating copies of original data

train_ = train.copy()
test_ = test.copy()

In [8]:
# Drop User Traffic

train.drop(["User_Traffic"],axis=1,inplace=True)

In [9]:
# Drop ID 

train.drop(["ID"],axis=1,inplace=True)
test.drop(["ID"],axis=1,inplace=True)

In [10]:
# Dealing with null values in the columns

train.fillna(train["Competition_Metric"].median(),inplace=True)
test.fillna(test["Competition_Metric"].median(),inplace=True)

In [11]:
# Dealing with Day Number - Changing it to Label Encoding

train["Day_No"] = train["Day_No"]%7
test["Day_No"] = test["Day_No"]%7

In [12]:
# Dealing with categorical Variables

enc = OneHotEncoder(dtype='int64',handle_unknown='ignore')

tr_val = enc.fit_transform(train[["Course_Domain","Course_Type"]]).toarray()
tr_df = pd.DataFrame(tr_val,columns=enc.get_feature_names())
train = pd.concat([train,tr_df],axis=1)
train.drop(["Course_Domain","Course_Type"],axis=1,inplace=True)

te_val = enc.transform(test[["Course_Domain","Course_Type"]]).toarray()
te_df = pd.DataFrame(te_val,columns=enc.get_feature_names())
test = pd.concat([test,te_df],axis=1)
test.drop(["Course_Domain","Course_Type"],axis=1,inplace=True)

In [ ]:
plt.subplots(figsize=(16,16))
sns.heatmap(train.corr(),annot=True, vmin=-0.1, vmax=0.1,center=0)

# Model Building

In [14]:
from sklearn.metrics import mean_squared_log_error as MSLE

In [15]:
# Metric for Model

def RMSLE(y, yhat):
    return (np.sqrt(MSLE(y, yhat)))*1000

def rmsle_xgb(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmsle", RMSLE(y,yhat)

In [16]:
# Defining the train, validate and test inputs and outputs

y = train.loc[:,"Sales"].copy()
X = train.loc[:,train.columns != 'Sales'].copy()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.05)

y_train = np.log1p(y_train)
y_val = np.log1p(y_val)

X_test = test.copy()

In [ ]:
# as the target variable is skewed applying log1p transformation

In [17]:
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.03,
          "max_depth": 14,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 10
          }
num_boost_round = 6000


dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_val, y_val)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

In [19]:
print("Train a XGBoost model")
start = time()
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, 
  early_stopping_rounds=50, feval=rmsle_xgb, verbose_eval=True)
end = time()
print('Training time is {:2f} s.'.format(end-start))

Train a XGBoost model
[0]	train-rmse:4.10447	eval-rmse:4.10372	train-rmsle:4104.43	eval-rmsle:4103.72
Multiple eval metrics have been passed: 'eval-rmsle' will be used for early stopping.

Will train until eval-rmsle hasn't improved in 50 rounds.
[1]	train-rmse:3.98229	eval-rmse:3.98157	train-rmsle:3982.29	eval-rmsle:3981.56
[2]	train-rmse:3.86396	eval-rmse:3.86322	train-rmsle:3863.98	eval-rmsle:3863.23
[3]	train-rmse:3.74895	eval-rmse:3.74818	train-rmsle:3748.94	eval-rmsle:3748.18
[4]	train-rmse:3.6375	eval-rmse:3.63676	train-rmsle:3637.5	eval-rmsle:3636.76
[5]	train-rmse:3.52942	eval-rmse:3.52868	train-rmsle:3529.41	eval-rmsle:3528.67
[6]	train-rmse:3.42449	eval-rmse:3.4238	train-rmsle:3424.49	eval-rmsle:3423.8
[7]	train-rmse:3.32272	eval-rmse:3.32201	train-rmsle:3322.71	eval-rmsle:3322.02
[8]	train-rmse:3.22413	eval-rmse:3.22347	train-rmsle:3224.14	eval-rmsle:3223.46
[9]	train-rmse:3.12849	eval-rmse:3.12785	train-rmsle:3128.5	eval-rmsle:3127.85
[10]	train-rmse:3.03561	eval-rmse:3.03

In [27]:
print("Make predictions on the test set")

dtest = xgb.DMatrix(X_test)
y_pred = gbm.predict(dtest)

# model1  kaggle private score 0.12647
pred_df = pd.DataFrame({"ID": test_['ID'], "Sales": np.expm1(y_pred)})
pred_df

Make predictions on the test set


,ID,Sales
0,883,115.034233
1,884,118.384979
2,885,100.410622
3,886,93.212082
4,887,51.281963
...,...,...
35995,548083,148.505936
35996,548084,120.826859
35997,548085,116.875389
35998,548086,118.406387


In [30]:
pred_df.to_csv("/content/xgb_submission.csv",index=False)

In [ ]:
# print("weight correction")
# W=[(0.995+(i/1000)) for i in range(30)]
# S =[]
# for w in W:
#     error = RMSLE(np.expm1(y_val), np.expm1(yhat*w))
#     print('RMSPE for {:.3f}:{:.6f}'.format(w,error))
#     S.append(error)
# Score = pd.Series(S,index=W)
# Score.plot()
# BS = Score[Score.values == Score.values.min()]
# print ('Best weight for Score:{}'.format(BS))

In [ ]:
# result_w3 = pd.DataFrame({"ID": test1['ID'], 'Sales': np.expm1(test_probs*1.001)})
# result_w3 #.to_csv('gbmm7.csv',index=False) 

In [ ]:
# out of the different models XGBoost Worked well and also Random Forest gave relatively good results.
# The predictions can be more improved provided there are more features
# Features like Year and Week of year were added and different models are builded but none of them performed better.Hence they are removed and only dy of week feature is added
# All the features were important and ID feature has much correlation with Course_ID variable hence it is dropped
